In [ ]:
import scanpy as sc
import scvelo as scv
import numpy as np

In [ ]:
adata = sc.read_h5ad("../../outputs_pancreas_4/pancreas/K11/adata/adata_K11_dt_ve.h5ad")

In [ ]:
import numpy as np

def least_squares_slope(time_vector, quantity_vector):
    """
    Computes the slope of the least squares line for the given time and quantity vectors.
    
    Parameters:
    time_vector (numpy.ndarray): The input time vector.
    quantity_vector (numpy.ndarray): The input quantity vector.
    
    Returns:
    float: The slope of the least squares line.
    """
    n = len(time_vector)
    sum_x = np.sum(time_vector)
    sum_y = np.sum(quantity_vector)
    sum_xy = np.sum(time_vector * quantity_vector)
    sum_x_squared = np.sum(time_vector ** 2)
    
    numerator = n * sum_xy - sum_x * sum_y
    denominator = n * sum_x_squared - sum_x ** 2
    
    if denominator == 0:
        raise ValueError("Denominator is zero, cannot compute slope.")
    
    slope = numerator / denominator
    return slope

In [ ]:
manifold = sc.read_h5ad("../../outputs/final_anndatas/pancreas/manifold.h5ad")
adata.obs["z5_pseudotime"] = manifold.obs["z5_pseudotime"].copy()

In [ ]:
import pandas as pd
adata.layers["slope_u"] = np.zeros(adata.shape)
adata.layers["slope_s"] = np.zeros(adata.shape)
for ctype in pd.unique(adata.obs["clusters"]):
    print(f"computing slopes for ctype: {ctype}..")
    ctype_obs = np.where(adata.obs["clusters"] == ctype)[0]
    for i,gene in enumerate(list(adata.var_names)):
        Mu = adata.layers["Mu"]
        Ms = adata.layers["Ms"]
        quantity_vector_u = Mu[ctype_obs,i]
        quantity_vector_s = Ms[ctype_obs,i]
        time_vector = adata.obs["z5_pseudotime"][ctype_obs]
        slope_u = least_squares_slope(time_vector, quantity_vector_u)
        slope_s = least_squares_slope(time_vector, quantity_vector_s)
        adata.layers["velocity_u"][ctype_obs, i] = slope_u
        adata.layers["velocity"][ctype_obs, i] = slope_s

In [ ]:
sc.pp.neighbors(adata)
scv.tl.velocity_graph(adata)
adata.write_h5ad("baseline.h5ad")